In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

In [ ]:
img_rows = 256
img_cols = 256
channels = 1
img_shape = (img_rows, img_cols, channels)
batch_size = 32

In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(128, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256 * 256 * 1, activation='tanh'))
    model.add(Reshape((256, 256, 1)))
    return model

In [ ]:

def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(256, 256, 1)))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [ ]:

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
discriminator.trainable = False
generator = build_generator()
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
# load data set from /dataset/


train_ds_skeleton = tf.keras.preprocessing.image_dataset_from_directory(
    '/dataset/skeleton/',
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=False,
    seed=123)

val_ds_skeleton = tf.keras.preprocessing.image_dataset_from_directory(
    '/dataset/skeleton/',
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=False,
    seed=123)

train_ds_animal = tf.keras.preprocessing.image_dataset_from_directory(
    '/dataset/animal/',
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=False,
    seed=123)

val_ds_animal = tf.keras.preprocessing.image_dataset_from_directory(
    '/dataset/animal/',
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=False,
    seed=123)



In [ ]:
storeloss = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    
    # labels
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):
        slice_start = (iteration * batch_size) % len(train_ds)
        slice_end = (iteration + batch_size) % len(train_ds)
        idx = train_ds[slice_start:slice_end]
        imgs = 